In [ ]:
# =============================================
# MÓDULO_06: Evaluación del Modelo Fine-Tuned
# =============================================

# ---------------------------------------------
# 1. Configuración Inicial del Entorno
# ---------------------------------------------
nombre_lote = "LOTE_20250614"
nombre_modulo = "MODULO_06"

try:
    import google.colab
    EN_COLAB = True
except ImportError:
    EN_COLAB = False

if EN_COLAB:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)
    ruta_base = "/content/drive/MyDrive/TFM_EVA_MARTIN/Modulos"
else:
    ruta_base = "G:/Mi unidad/TFM_EVA_MARTIN/Modulos"

import os
import sys
import logging

if ruta_base not in sys.path:
    sys.path.append(ruta_base)

import utilidades_comunes

# Cargar configuración YAML
import yaml
ruta_config = os.path.join(ruta_base, "config.yaml")
with open(ruta_config, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)
params = config["parametros"]

# Configurar entorno y logger
logger = utilidades_comunes.configurar_logger(nombre_modulo)
logger.setLevel(logging.DEBUG)
entorno = utilidades_comunes.inicializar_entorno(nombre_modulo, nombre_lote, ruta_base)

# ---------------------------------------------
# 2. Cargar modelo final y tokenizer
# ---------------------------------------------
from transformers import LongformerTokenizerFast, LongformerForSequenceClassification
import torch

modelo_dir = os.path.join(ruta_base, "MODULO_05", "salida", "modelo_final")
tokenizer = LongformerTokenizerFast.from_pretrained(modelo_dir)
model = LongformerForSequenceClassification.from_pretrained(modelo_dir)
model.eval()

# ---------------------------------------------
# 3. Cargar conjunto de prueba
# ---------------------------------------------
test_texts = [...]  # Sustituir con textos reales de test
test_labels = [...] # Sustituir con etiquetas codificadas (0, 1, 2)

# Tokenizar
inputs = tokenizer(test_texts, truncation=True, padding=True, return_tensors="pt")

class TFM_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

test_dataset = TFM_Dataset(inputs, test_labels)

# ---------------------------------------------
# 4. Evaluación y métricas
# ---------------------------------------------
from transformers import Trainer
from sklearn.metrics import classification_report

trainer = Trainer(model=model)

logger.info("🧪 Realizando predicción sobre el conjunto de prueba…")
predictions = trainer.predict(test_dataset)
y_pred = predictions.predictions.argmax(axis=-1)
y_true = predictions.label_ids

# Reporte detallado
report = classification_report(y_true, y_pred, target_names=["Positivo", "Neutro", "Negativo"], output_dict=True)

# Mostrar como tabla
import pandas as pd
report_df = pd.DataFrame(report).transpose()
print(report_df)

# Guardar como CSV
salida_csv = os.path.join(entorno["ruta_salida"], "evaluacion_metrica_detallada.csv")
report_df.to_csv(salida_csv, index=True, encoding="utf-8")
logger.info(f"📄 Reporte de evaluación guardado en: {salida_csv}")

logger.info(f"✅ Evaluación finalizada correctamente")
